# Tugas 4: Antarmuka Percakapan - Mengobrol dengan Llama 3 dan Titan Premier LLM

Di <i>notebook</i> ini, Anda membuat <i>chatbot</i> menggunakan Model Fondasi (FM) llama3-8b-instruct dan titan-text-premier di Amazon Bedrock.

Antarmuka percakapan seperti <i>chatbot</i> dan asisten virtual dapat meningkatkan pengalaman pengguna bagi pelanggan Anda. <i>Chatbot</i> menggunakan pemrosesan bahasa alami (NLP) dan algoritma <i>machine learning</i> untuk memahami dan merespons pertanyaan pengguna. Anda dapat menggunakan <i>chatbot</i> di berbagai aplikasi, seperti layanan pelanggan, penjualan, dan <i>e-commerce</i>, untuk memberikan respons yang cepat dan efisien kepada pengguna. Pengguna dapat mengaksesnya melalui berbagai saluran seperti situs web, platform media sosial, dan aplikasi perpesanan.

- **<i>Chatbot</i> (Dasar)** - <i>Chatbot Zero Shot</i> dengan model FM
- **<i>Chatbot</i> menggunakan perintah** - template(LangChain) - <i>Chatbot</i> dengan beberapa konteks yang diberikan pada templat perintah
- **<i>Chatbot</i> dengan persona** - <i>Chatbot</i> dengan peran yang ditentukan, yaitu interaksi Pelatih Karier dan Manusia
- **<i>Chatbot</i> sadar konteks** - Meneruskan konteks melalui <i>file</i> eksternal dengan membuat penyematan.

## Kerangka kerja LangChain untuk membuat <i>Chatbot</i> dengan Amazon Bedrock

Dalam antarmuka percakapan seperti <i>chatbot</i>, sangat penting untuk mengingat interaksi sebelumnya, baik dalam jangka pendek atau dalam jangka panjang.

Kerangka kerja LangChain menyediakan komponen memori dalam dua bentuk. Pertama, LangChain menyediakan utilitas pembantu untuk mengelola dan memanipulasi pesan obrolan sebelumnya. Ini dirancang untuk menjadi modular. Kedua, LangChain menyediakan cara mudah untuk menggabungkan utilitas ini ke dalam <i>chain</i>, sehingga Anda dapat dengan mudah mendefinisikan dan berinteraksi dengan berbagai jenis abstraksi, yang membuat <i>chatbot</i> canggih mudah untuk Anda buat.

## Membuat <i>Chatbot</i> dengan Konteks - Elemen Utama

Proses pertama dalam membangun <i>chatbot</i> sadar konteks adalah membuat penyematan untuk konteks. Biasanya, ada proses <i>ingestion</i> yang dijalankan melalui model penyematan Anda dan menghasilkan penyematan yang akan disimpan di penyimpanan vektor. Di <i>notebook </i>ini, Anda menggunakan model Titan Embeddings untuk ini. Proses kedua adalah orkestrasi permintaan pengguna, interaksi, pemanggilan, dan pengembalian hasil. Hal ini melibatkan pengaturan permintaan pengguna, berinteraksi dengan model/komponen yang diperlukan untuk mengumpulkan informasi, memanggil <i>chatbot </i>untuk merumuskan respons, dan kemudian mengembalikan respons <i>chatbot </i>kepada pengguna.

## Tugas 4.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Tugas 4.2: Menggunakan riwayat obrolan dari LangChain untuk memulai percakapan

Dalam tugas ini, Anda mengaktifkan <i>chatbot</i> untuk membawa konteks percakapan di beberapa interaksi dengan pengguna. Memiliki memori percakapan sangat penting bagi <i>Chatbot</i> untuk melakukan dialog yang bermakna dan koheren dari waktu ke waktu.

Anda menerapkan kemampuan memori percakapan dengan membuat di atas kelas InMemoryChatMessageHistory LangChain. Objek ini menyimpan percakapan antara pengguna dan <i>chatbot</i>, dan riwayat tersedia di agen <i>chatbot</i> sehingga dapat memanfaatkan konteks dari percakapan sebelumnya.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Catatan:** <i>Output</i> model bersifat non-deterministik.

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### Pertanyaan Baru

Model telah merespons dengan pesan awal. Sekarang, Anda mengajukan beberapa pertanyaan.

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Mengembangkan pertanyaan

Sekarang, ajukan pertanyaan tanpa menyebutkan kata taman untuk melihat apakah model dapat memahami percakapan sebelumnya.

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### Menyelesaikan percakapan ini

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## Tugas 4.3: <i>Chatbot</i> menggunakan templat perintah (LangChain)

Dalam tugas ini, Anda menggunakan PromptTemplate <i>default</i> yang bertanggung jawab atas konstruksi input ini. LangChain menyediakan beberapa kelas dan fungsi untuk memudahkan pembuatan dan pengerjaan perintah.

In [ ]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Selanjutnya, mulai obrolan.

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## Tugas 4.4: <i>Chatbot</i> dengan persona

Dalam tugas ini, asisten Kecerdasan Buatan (AI) berperan sebagai pelatih karier. Anda dapat memberi tahu <i>chatbot</i> tentang persona (atau peran) menggunakan pesan sistem. Terus manfaatkan kelas InMemoryChatMessageHistory untuk mempertahankan konteks percakapan.

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

Sekarang, ajukan pertanyaan yang tidak termasuk dalam spesialisasi persona ini. Model seharusnya tidak menjawab pertanyaan itu dan harus memberikan alasan untuk itu.

## Task 4.5 <i>Chatbot</i> dengan Konteks

Dalam tugas ini, Anda meminta <i>chatbot</i> untuk menjawab pertanyaan berdasarkan konteks yang diteruskan kepadanya. Anda mengambil <i>file</i> CSV dan menggunakan model penyematan Titan untuk membuat vektor yang mewakili konteks itu. Vektor ini disimpan di Facebook AI Similarity Search (FAISS). Ketika <i>chatbot </i>diberikan pertanyaan, Anda akan meneruskan vektor ini kembali ke <i>chatbot </i>dan meminta ia mengambil jawabannya menggunakan vektor.

### Model Titan Embeddings

Penyematan merepresentasikan kata, frasa, atau item diskrit lainnya sebagai vektor dalam ruang vektor berkelanjutan. Hal ini memungkinkan model <i>machine learning</i> melakukan operasi matematika pada representasi ini dan menangkap hubungan semantik di antara representasi tersebut.

Anda menggunakan penyematan untuk <i>Retrieval-Augmented Generation</i> (RAG) [kemampuan pencarian dokumen](https://labelbox.com/blog/how-vector-similarity-search-works/).

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS sebagai VectorStore

Agar dapat menggunakan penyematan untuk pencarian, Anda memerlukan penyimpanan yang dapat melakukan pencarian kesamaan vektor secara efisien. Di <i>notebook</i> ini, Anda menggunakan FAISS, yang merupakan penyimpanan memori. Untuk menyimpan vektor secara permanen, Anda dapat menggunakan Basis Pengetahuan untuk Amazon Bedrock, pgVector, Pinecone, Weaviate, atau Chroma.

API LangChain VectorStore tersedia [di sini](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### Jalankan pengujian kode rendah cepat 

Anda dapat menggunakan kelas <i>Wrapper</i> yang disediakan oleh LangChain untuk menanyakan penyimpanan basis data vektor dan mengembalikan dokumen yang relevan. Kelas <i>wrapper</i> ini menjalankan QA Chain dengan semua nilai <i>default</i>.

In [ ]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### Aplikasi <i>chatbot</i>

Untuk <i>chatbot</i>, Anda memerlukan manajemen konteks, riwayat, penyimpanan vektor, dan banyak hal lainnya. Anda mulai dengan membangun <i>chain Retrieval Augmented Generation</i> (RAG) yang mendukung konteks.

Ini menggunakan fungsi **create_stuff_documents_chain** dan **create_retrieval_chain**

### Parameter dan fungsi yang digunakan untuk RAG

- **Retriever:** Anda menggunakan `VectorStoreRetriever`, yang didukung oleh `VectorStore`. Untuk mengambil teks, ada dua jenis pencarian yang dapat dipilih: `"similarity"` atau `"mmr"`. `search_type="similarity"` menggunakan pencarian kesamaan dalam objek pengambilan yang memilih vektor potongan teks yang paling mirip dengan vektor pertanyaan.

- **create_stuff_documents_chain** menentukan bagaimana konteks yang diambil dimasukkan ke dalam perintah dan LLM. Dokumen yang diambil "diisi" sebagai konteks tanpa ringkasan atau pemrosesan lain ke dalam perintah.

- **create_retrieval_chain** menambahkan langkah pengambilan dan menyebarkan konteks melalui <i>chain</i>, menyediakannya di samping jawaban akhir. 

Jika pertanyaan yang diajukan berada di luar cakupan konteks, model akan membalas tidak mengetahui jawabannya.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

Selanjutnya, mulai obrolan

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


Anda telah menggunakan Titan LLM untuk membuat antarmuka percakapan dengan pola berikut:

- <i>Chatbot</i> (Dasar - tanpa konteks)
- <i>Chatbot</i> menggunakan templat perintah (Langchain)
- <i>Chatbot</i> dengan persona
- <i>Chatbot</i> dengan konteks

### Cobalah sendiri

- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini dan lanjutkan dengan **Tugas 5**.